## 🗜 Flatten!
Sometimes deeply nested commits can be annoying ...

In [ ]:
# Import required libraries
from specklepy.api.client import SpeckleClient
from specklepy.core.api import operations
from specklepy.transports.server import ServerTransport
from IPython.display import IFrame

# Define global variables 
HOST = "https://app.speckle.systems/"
AUTHENTICATION_TOKEN = ""
STREAM_ID = ""
BRANCH_NAME = ""

# Setting up SpeckleClient and authenticating
client = SpeckleClient(host=HOST)
client.authenticate_with_token(token=AUTHENTICATION_TOKEN)

# Receving commit
transport = ServerTransport(STREAM_ID, client)
branch = client.branch.get(stream_id=STREAM_ID, name=BRANCH_NAME)
model_data = operations.receive(branch.commits.items[0].referencedObject, transport)


In [ ]:
"""Helper module for a simple speckle object tree flattening."""

from collections.abc import Iterable
from typing import Any

from specklepy.objects import Base


def flatten_base(base: Base) -> Iterable[Base]:
    """Flatten a base object into an iterable of bases.
    
    This function recursively traverses any attribute that starts with '@' in the
    base object, yielding each nested base object found within those attributes.

    Args:
        base (Base): The base object to flatten.

    Yields:
        Base: Each nested base object in the hierarchy.
    """
    def _get_elements_from_attr(obj: Any) -> list[Base]:
        """Helper function to get Base objects from an attribute value."""
        if isinstance(obj, (list, tuple)):
            return [item for item in obj if isinstance(item, Base)]
        elif isinstance(obj, Base):
            return [obj]
        return []

    # Get all attributes that start with '@'
    at_attributes = [attr for attr in dir(base) if attr.startswith('@')]
    
    # Process each @ attribute
    for attr in at_attributes:
        elements = getattr(base, attr, None)
        if elements is not None:
            # Get all Base objects from this attribute
            base_elements = _get_elements_from_attr(elements)
            # Recursively process each Base object
            for element in base_elements:
                yield from flatten_base(element)
    
    yield base

In [ ]:
flattened_base = flatten_base(model_data)
for object in flattened_base:
    print(object)
    break

In [ ]:
flatten_base = flatten_base(model_data)
for object in flatten_base:
    print(object)
    break